In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Paren Checker

In [9]:
from typing import Callable
import torch
from cases.paren_checker import HighLevelParensBalanceChecker, test_HL_parens_balancer_components, BalancedParensDataset

In [14]:
test_HL_parens_balancer_components()

All Balance tests passed!


True

In [15]:
hl_model = HighLevelParensBalanceChecker()
corr = hl_model.get_correspondence()
dataset = BalancedParensDataset(N_samples=5_000, n_ctx=hl_model.get_ll_model_cfg().n_ctx, seed=42)
train_set, test_set = dataset.get_IIT_train_test_set()

making IIT dataset


In [16]:
print(dataset.get_dataset().shape)
print(dataset.get_dataset()[:10]['tokens'])
print(dataset.get_dataset()[:10]['labels'])
for i in range(10):
    tokens, labels, hl_outputs = dataset.get_dataset()[i]['tokens'], dataset.get_dataset()[i]['labels'], hl_model((torch.tensor(dataset.get_dataset()[i]['tokens'])[None,:], None, None))
    nonzero = (torch.tensor(labels) - hl_outputs[0]).nonzero()
    if nonzero.numel() > 0:
        print(tokens, torch.unique(nonzero[:,0]))
        bad_indices = torch.unique(nonzero[:,0]).tolist()
        for idx in bad_indices:
            print(labels[idx], hl_outputs[0,idx])

(5000, 4)
[[3, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 2], [3, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2], [3, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2], [3, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 2], [3, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 2], [3, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 2], [3, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 2], [3, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 2], [3, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 2], [3, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 2]]
[[[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0],

In [17]:
from iit.model_pairs.strict_iit_model_pair import StrictIITModelPair
from iit.utils.index import Ix
import torch

n_epochs = 100
training_args = {
    "batch_size": 256,
    "lr": 0.001,
    "num_workers": 0,
    "use_single_loss": False,
    "behavior_weight": 0., #basically doubles the strict weight's job.
    "iit_weight": 1.,
    "strict_weight": 1.0,
    "clip_grad_norm": 1.0,
    "iit_weight_schedule" : lambda s, i: s,
    "strict_weight_schedule" : lambda s, i: s,
    "behavior_weight_schedule" : lambda s, i: s, #0.955*s if 0.955**i > 0.01 else s, #have behavior weight decay over time
    "early_stop" : True,
    "lr_scheduler": torch.optim.lr_scheduler.LinearLR,
    "scheduler_kwargs": dict(start_factor=1, end_factor=0, total_iters=n_epochs),
    "scheduler_val_metric": ["val/accuracy", "val/IIA"], #for ReduceLRonPlateau
    "scheduler_mode": "max", #for ReduceLRonPlateau
}

class LastTokenStrictIITModelPair(StrictIITModelPair):
    def get_label_idxs(self):
        return Ix[[None,-1]]

    # def loss_fn(self) -> Callable[[torch.Tensor, torch.Tensor], torch.Tensor]:
    #     return torch.nn.CrossEntropyLoss()

ll_model = hl_model.get_ll_model()
model_pair = StrictIITModelPair(hl_model=hl_model, ll_model=ll_model, corr=corr, training_args=training_args)

In [19]:
model_pair.train(
    train_set=train_set,
    test_set=test_set,
    optimizer_cls=torch.optim.AdamW,
    epochs=n_epochs,
    optimizer_kwargs=dict(weight_decay=1e-4),
)

training_args={'batch_size': 256, 'lr': 0.001, 'num_workers': 0, 'early_stop': True, 'lr_scheduler': <class 'torch.optim.lr_scheduler.LinearLR'>, 'scheduler_val_metric': ['val/accuracy', 'val/IIA'], 'scheduler_mode': 'max', 'scheduler_kwargs': {'start_factor': 1, 'end_factor': 0, 'total_iters': 100}, 'clip_grad_norm': 1.0, 'seed': 0, 'detach_while_caching': True, 'iit_weight_schedule': <function <lambda> at 0x17f69c220>, 'strict_weight_schedule': <function <lambda> at 0x17f7580e0>, 'behavior_weight_schedule': <function <lambda> at 0x17f758ae0>, 'atol': 0.05, 'use_single_loss': False, 'iit_weight': 1.0, 'behavior_weight': 0.0, 'strict_weight': 1.0}


Epoch 1: lr: 9.90e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 4.66e-01, train/behavior_loss: 0.00e+00, train/strict_loss: 3.96e-01, val/iit_loss: 1.70e-01, val/IIA: 95.48, val/accuracy: 95.57, val/strict_accuracy: 95.57
Epoch 2: lr: 9.80e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 1.61e-01, train/behavior_loss: 0.00e+00, train/strict_loss: 9.22e-02, val/iit_loss: 1.43e-01, val/IIA: 95.29, val/accuracy: 99.07, val/strict_accuracy: 99.07
Epoch 3: lr: 9.70e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 1.67e-01, train/behavior_loss: 0.00e+00, train/strict_loss: 7.08e-02, val/iit_loss: 9.62e-02, val/IIA: 97.06, val/accuracy: 98.95, val/strict_accuracy: 98.95
Epoch 4: lr: 9.60e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 9.81e-02, train/behavior_loss: 0.00e+00, train/strict_loss: 3.27e-02, val/

# Left > Right

In [20]:
from cases.left_greater import HighLevelLeftGreater, test_HL_left_greater_components, LeftGreaterDataset

test_HL_left_greater_components()

All left greater tests passed!


True

In [21]:
hl_model = HighLevelLeftGreater()
corr = hl_model.get_correspondence()
dataset = LeftGreaterDataset(N_samples=1_000, n_ctx=hl_model.get_ll_model_cfg().n_ctx, seed=42)
train_set, test_set = dataset.get_IIT_train_test_set()
print(hl_model.hook_dict)
print(list(corr.keys()))

making IIT dataset
{'input_hook': HookPoint(), 'paren_counts_hook': HookPoint(), 'mlp0_hook': HookPoint()}
[input_hook, paren_counts_hook, mlp0_hook]


In [22]:
ll_model = hl_model.get_ll_model()
model_pair = StrictIITModelPair(hl_model=hl_model, ll_model=ll_model, corr=corr, training_args=training_args)
model_pair.train(
    train_set=train_set,
    test_set=test_set,
    optimizer_cls=torch.optim.AdamW,
    epochs=n_epochs,
    optimizer_kwargs=dict(weight_decay=1e-4),
)

training_args={'batch_size': 256, 'lr': 0.001, 'num_workers': 0, 'early_stop': True, 'lr_scheduler': <class 'torch.optim.lr_scheduler.LinearLR'>, 'scheduler_val_metric': ['val/accuracy', 'val/IIA'], 'scheduler_mode': 'max', 'scheduler_kwargs': {'start_factor': 1, 'end_factor': 0, 'total_iters': 100}, 'clip_grad_norm': 1.0, 'seed': 0, 'detach_while_caching': True, 'iit_weight_schedule': <function <lambda> at 0x17f69c220>, 'strict_weight_schedule': <function <lambda> at 0x17f7580e0>, 'behavior_weight_schedule': <function <lambda> at 0x17f758ae0>, 'atol': 0.05, 'use_single_loss': False, 'iit_weight': 1.0, 'behavior_weight': 0.0, 'strict_weight': 1.0}


Epoch 1: lr: 9.90e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 1.32e+00, train/behavior_loss: 0.00e+00, train/strict_loss: 1.27e+00, val/iit_loss: 1.08e+00, val/IIA: 55.73, val/accuracy: 55.80, val/strict_accuracy: 55.80
Epoch 2: lr: 9.80e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 9.92e-01, train/behavior_loss: 0.00e+00, train/strict_loss: 9.35e-01, val/iit_loss: 1.02e+00, val/IIA: 46.99, val/accuracy: 65.12, val/strict_accuracy: 65.12
Epoch 3: lr: 9.70e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 8.96e-01, train/behavior_loss: 0.00e+00, train/strict_loss: 7.78e-01, val/iit_loss: 9.46e-01, val/IIA: 46.30, val/accuracy: 67.11, val/strict_accuracy: 67.11
Epoch 4: lr: 9.60e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 7.17e-01, train/behavior_loss: 0.00e+00, train/strict_loss: 7.05e-01, val/

# Duplicate remover
case 19 in circuits-bench

In [49]:
from cases.duplicate_remover import HighLevelDuplicateRemover, test_HL_duplicate_remover_components, DuplicateRemoverDataset
test_HL_duplicate_remover_components()

[[4, 0, 0, 1, 2, 0, 1, 3, 3], [4, 0, 1, 2, 2, 2, 2, 2, 2], [4, 0, 1, 2, 3, 3, 3, 3, 3]]
[[False, False, True, False, False, False, False, False, True], [False, False, False, False, True, True, True, True, True], [False, False, False, False, False, True, True, True, True]]
All DuplicateRemover tests passed!


True

In [59]:
hl_model = HighLevelDuplicateRemover()
corr = hl_model.get_correspondence()
dataset = DuplicateRemoverDataset(N_samples=1_000, n_ctx=hl_model.get_ll_model_cfg().n_ctx, seed=42)
train_set, test_set = dataset.get_IIT_train_test_set()
print(hl_model.hook_dict)
print(list(corr.keys()))

(1000, 15, 5)
making IIT dataset
{'input_hook': HookPoint(), 'prev_token_hook': HookPoint(), 'prev_equal_hook': HookPoint(), 'output_hook': HookPoint()}
[input_hook, prev_token_hook, prev_equal_hook, output_hook]


In [63]:
ll_model = hl_model.get_ll_model()
model_pair = StrictIITModelPair(hl_model=hl_model, ll_model=ll_model, corr=corr, training_args=training_args)
model_pair.train(
    train_set=train_set,
    test_set=test_set,
    optimizer_cls=torch.optim.AdamW,
    epochs=n_epochs,
    optimizer_kwargs=dict(weight_decay=1e-4),
)

training_args={'batch_size': 256, 'lr': 0.001, 'num_workers': 0, 'early_stop': True, 'lr_scheduler': <class 'torch.optim.lr_scheduler.LinearLR'>, 'scheduler_val_metric': ['val/accuracy', 'val/IIA'], 'scheduler_mode': 'max', 'scheduler_kwargs': {'start_factor': 1, 'end_factor': 0, 'total_iters': 100}, 'clip_grad_norm': 1.0, 'seed': 0, 'detach_while_caching': True, 'iit_weight_schedule': <function <lambda> at 0x17f69c220>, 'strict_weight_schedule': <function <lambda> at 0x17f7580e0>, 'behavior_weight_schedule': <function <lambda> at 0x17f758ae0>, 'atol': 0.05, 'use_single_loss': False, 'iit_weight': 1.0, 'behavior_weight': 0.0, 'strict_weight': 1.0}


Epoch 1: lr: 9.90e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 1.62e+00, train/behavior_loss: 0.00e+00, train/strict_loss: 1.55e+00, val/iit_loss: 1.33e+00, val/IIA: 43.05, val/accuracy: 44.02, val/strict_accuracy: 44.02
Epoch 2: lr: 9.80e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 1.42e+00, train/behavior_loss: 0.00e+00, train/strict_loss: 1.20e+00, val/iit_loss: 1.42e+00, val/IIA: 35.78, val/accuracy: 61.51, val/strict_accuracy: 61.51
Epoch 3: lr: 9.70e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 1.18e+00, train/behavior_loss: 0.00e+00, train/strict_loss: 9.54e-01, val/iit_loss: 1.39e+00, val/IIA: 35.54, val/accuracy: 69.45, val/strict_accuracy: 69.45
Epoch 4: lr: 9.60e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 1.01e+00, train/behavior_loss: 0.00e+00, train/strict_loss: 7.84e-01, val/

# Unique token extraction
case 21 in circuits-bench

In [48]:
import torch
print(torch.randint(0, 3, (10, 2)))

a = [1, 2, 3]
a.remove(2)
print(a)

tensor([[2, 2],
        [2, 1],
        [0, 0],
        [1, 2],
        [0, 2],
        [2, 0],
        [2, 2],
        [0, 2],
        [1, 0],
        [2, 0]])
[1, 3]


# Increasing / Decreasing detector
case 13 in circuits-bench